<a href="https://colab.research.google.com/github/shivamsouravjha/Toonification/blob/main/generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from tensorflow import keras
import cv2
from keras.models import Model
from keras.callbacks import TensorBoard

from keras.models import load_model
import math
import numpy as np
import pathlib
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops
from keras.models import Sequential # to create a cnn model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,UpSampling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop,Adam,SGD,Adagrad,Adadelta,Adamax,Nadam
from keras.applications import xception
from keras.layers import LeakyReLU


In [32]:
def boxx(a ,k,n,s):
  x1 = Conv2D(n, (k ,1), padding='same',strides=(s,s))(a)
  x1 = LeakyReLU()(x1)
  return x1, x1

In [33]:
def boxy(a,b,k,n,s):
  x1 = Conv2D(n, (k ,1), padding='same',strides=(s,s))(a)
  x1 = LeakyReLU()(x1)
  x1 = Conv2D(n, (k ,1), padding='same',strides=(s,s))(x1)
  x1 = Add()([x1,b])
  return x1, x1


In [34]:
input_img = Input(shape=(64,64,1))



In [35]:
a0,b0  = boxx(input_img,7,32,1)  #block 1

In [36]:
a1 = boxx(a0,3,32,2)[0]            #block 2(a)
a2,b2 = boxx(a1,3,64,1)             #block 2(B)

In [39]:
a3 = boxx(a2,3,32,2)[0]            #block 3(a)
a4,b4 = boxx(a3,3,128,1)             #block 3(B)

In [40]:
 a5,b5 = boxy(a3,a4,3,128,1)          #block 4

In [ ]:
 a6,b6 = boxy(a5,a5,3,128,1)          #block 5

In [ ]:
 a7,b7 = boxy(a6,a6,3,128,1)          #block 6

In [ ]:
 a8,b8 = boxy(a7,a7,3,128,1)          #block 7

In [ ]:
a3 = boxx(a2,3,32,2)[0]            #block 3(a)
a4,b4 = boxx(a3,3,64,1)             #block 3(B)

In [ ]:
a3 = boxx(a2,3,32,2)[0]            #block 3(a)
a4,b4 = boxx(a3,3,64,1)             #block 3(B)

In [ ]:
autoencoder =Model(input_img,boxx(input_img,7,32,1)[0])

In [ ]:
autoencoder = Model(input_img, x1)

In [ ]:

from tensorflow.keras.utils import plot_model
plot_model(autoencoder, to_file='model.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image("model.png")